# Cellchat fetch resuls

In [ ]:
options(warn=-1, stringsAsFactors=FALSE)

In [ ]:
library_load <- suppressMessages(
    
    list(
        
        # CellTalk
        library(CellChat), 
        
        # Seurat 
        library(Seurat), 
        
        # Data 
        library(tidyverse), 
        
        # Plotting 
        library(ComplexHeatmap), 
        library(circlize), 
        library(viridis), 
        library(ggplotify), 
        library(ggrepel), 
        library(cowplot)
        
    )
)

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
ht_opt$message=FALSE # ComplexHeatmap 

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source files
source("plotting_global.R")
source("bin/cci_pp.R")
source("bin/cci_pl.R")

In [ ]:
# Plotting Theme
ggplot2::theme_set(theme_global_set(1)) # From project global source()

# Parameter and files 

In [ ]:
so_file <- "data/scRNAseq/object/pp.rds"

# Import 

In [ ]:
so <- readRDS(so_file)
so <- NormalizeData(so)

# Parameter 

In [ ]:
pval_prob_thr <- 0.05
p_val_adj_dea <- 0.05

# CellChat results

In [ ]:
lr_res <- read.csv("result/cci/lr_res.csv", row.names=1)

## Add reversed Cell-cell interactions 

In [ ]:
# Load full data base 
db <- CellChat::CellChatDB.mouse

In [ ]:
# Impute custom interactions
options(stringsAsFactors=FALSE)
interaction <- read.csv("data/reference/cellchat/interaction.csv", row.names=1, na.strings=c(""))
interaction[is.na(interaction)] <- ""
db$interaction <- rbind(db$interaction, interaction)

In [ ]:
# Get cell-cell contact interaction 
db <- CellChat::subsetDB(db, search=c("Cell-Cell Contact"), key="annotation")

In [ ]:
# Subset lr results for cell-cell contact forward 
lr_res_ccc_fwd <- lr_res %>% dplyr::filter(lr_res$interaction_name %in% db$interaction$interaction_name)

In [ ]:
# Reverse cell-cell contact 
lr_res_ccc_rev <- data.frame(

    source=lr_res_ccc_fwd$target, 
    target=lr_res_ccc_fwd$source, 
    interaction_name=paste0(lr_res_ccc_fwd$receptor_symbol, "_", lr_res_ccc_fwd$ligand_symbol), 
    prob=lr_res_ccc_fwd$prob, 
    pval=lr_res_ccc_fwd$pval, 
    pathway_name=lr_res_ccc_fwd$pathway_name, 
    ligand_symbol=lr_res_ccc_fwd$receptor_symbol, 
    receptor_symbol=lr_res_ccc_fwd$ligand_symbol, 
    sample_group=lr_res_ccc_fwd$sample_group
    
)

In [ ]:
# Combine results 
lr_res_ccc <- rbind(lr_res_ccc_fwd, lr_res_ccc_rev)

In [ ]:
# Assemble LR results 
lr_res <- rbind(lr_res[!lr_res$interaction_name %in% lr_res_ccc$interaction_name, ], lr_res_ccc)

# Parameter 

In [ ]:
pval_prob_thr <- 0.05
p_val_adj_dea <- 0.01

In [ ]:
pathway_name_thr <- c("ICAM", "VCAM", "SIRP", "KIT", "TGFb", "TNF", "CXCL", "CCL")

In [ ]:
lr_res <- lr_res[lr_res$pathway_name %in% pathway_name_thr, ]

In [ ]:
lr_res$pathway_name <- factor(lr_res$pathway_name, levels=pathway_name_thr)

# DEA results 

In [ ]:
dea_res_1 <- readRDS("result/dea/scRNAseq/wilcox/IFNAR_fl_CpG_D1_vs_IFNAR_fl_Baseline_D0.rds")[c("cMo", "intMo", "RPM", "cDC2", "Basophil", "MEP", "Proerythroblast", "Erythroblast")]
dea_res_1 <- lapply(dea_res_1, function(x) {list("D1"=x)})

In [ ]:
dea_res_2 <- readRDS("result/dea/scRNAseq/wilcox/IFNAR_fl_CpG_D1_vs_IFNAR_fl_Baseline_D0.rds")[c("cMo", "intMo", "RPM", "cDC2", "Basophil", "MEP", "Proerythroblast", "Erythroblast")]
dea_res_2 <- lapply(dea_res_2, function(x) {list("D3"=x)})

In [ ]:
dea_res <- Map(c, dea_res_1, dea_res_2)

## PTPG B (+/+) vs D1 (+/+) and D3 (+/+)

## cMo

In [ ]:
source <- c("cMo")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
options(repr.plot.width=5, repr.plot.height=12)

dea_lr_hm(lr_res_i, dea_res, source, target)

In [ ]:
source <- c("cMo")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
options(repr.plot.width=5, repr.plot.height=10)

dea_lr_hm(lr_res_i, dea_res, source, target)

In [ ]:
source <- c("cMo")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
options(repr.plot.width=5, repr.plot.height=10)

dea_lr_hm(lr_res_i, dea_res, source, target)

## intMo

In [ ]:
source <- c("intMo")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
options(repr.plot.width=5, repr.plot.height=12)

dea_lr_hm(lr_res_i, dea_res, source, target)

In [ ]:
source <- c("intMo")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
options(repr.plot.width=5, repr.plot.height=10)

dea_lr_hm(lr_res_i, dea_res, source, target)

In [ ]:
source <- c("intMo")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
options(repr.plot.width=5, repr.plot.height=8)

dea_lr_hm(lr_res_i, dea_res, source, target)

## RPM

In [ ]:
source <- c("RPM")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
options(repr.plot.width=5, repr.plot.height=15)

dea_lr_hm(lr_res_i, dea_res, source, target)

In [ ]:
source <- c("RPM")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
options(repr.plot.width=5, repr.plot.height=12)

dea_lr_hm(lr_res_i, dea_res, source, target)

In [ ]:
source <- c("RPM")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
options(repr.plot.width=5, repr.plot.height=10)

dea_lr_hm(lr_res_i, dea_res, source, target)

## cDC2

In [ ]:
source <- c("cDC2")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
options(repr.plot.width=5, repr.plot.height=10)

dea_lr_hm(lr_res_i, dea_res, source, target)

In [ ]:
source <- c("cDC2")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
options(repr.plot.width=5, repr.plot.height=10)

dea_lr_hm(lr_res_i, dea_res, source, target)

In [ ]:
source <- c("cDC2")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
options(repr.plot.width=5, repr.plot.height=10)

dea_lr_hm(lr_res_i, dea_res, source, target)

## Basophil

In [ ]:
source <- c("Basophil")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)

dea_lr_hm(lr_res_i, dea_res, source, target)

In [ ]:
source <- c("Basophil")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)

dea_lr_hm(lr_res_i, dea_res, source, target)

In [ ]:
source <- c("Basophil")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)

dea_lr_hm(lr_res_i, dea_res, source, target)